We retrieve the results obtained in `graph_fact.py`

In [1]:
import time
import numpy as np
import pickle
with open(r"c:\Users\cacereje\OneDrive\Vanderbilt\Postdoc 2025 Fall\Code\Graph_Fact\result.pkl", "rb") as f:
     result = pickle.load(f)

To determine the factored graph $G$, we just multiply all the matrices in any list of `results`.

In [2]:
import sympy as sp

G = sp.Matrix(np.linalg.multi_dot(result[0]))
G

Matrix([
[1, 0, 1, 0, 0, 0, 0, 0],
[0, 1, 0, 1, 0, 0, 0, 0],
[0, 0, 1, 1, 1, 1, 0, 0],
[0, 0, 0, 0, 1, 0, 1, 0],
[0, 0, 0, 0, 0, 1, 0, 1]])

Here $a_n$ and $s_n$ will be the dimension and trace vector of $A_{0n}$. This means that $s_n$ is a PF eigenvector of $GG^t$. Moreover, to have $\tau(1)=1$ we make sure that $\langle a_n, s_n\rangle =1$.

In [ ]:
## The functions below compute the dimension vector, trace vectors and norm for the bottom and intermediate inclusions.

a_0 = sp.ones(5,1)
b_0 = (G.T)*a_0
s_0_raw = (G*(G.T)).eigenvects()[-1][-1][0]
t_0_raw = ((G.T)*G).eigenvects()[-1][-1][0]
s_0 = s_0_raw/((a_0.T*s_0_raw)[0])
s_0 = sp.together(sp.simplify(s_0))
t_0 = t_0_raw/((b_0.T*t_0_raw)[0])
t_0 = sp.together(sp.simplify(t_0))

def a(n):
    return ((G*(G.T))**(n))*a_0 

def b(n):
    return (((G.T)*G)**(n))*b_0 

def s(n):
    s = s_0/((a(n).T*s_0)[0])
    return sp.together(sp.simplify(s))

def t(n):
    t = t_0/((b(n).T*t_0)[0])
    return sp.together(sp.simplify(t))

def norm_sym(A,n):
    sa = s(n)
    return sp.together(sp.simplify(sp.expand((A*(A.T)*sa).T*sa)/(((sa.T)*sa)[0])))[0]

def norm_sym2(A,n):
    ta = t(n)
    return sp.together(sp.simplify(sp.expand(((A.T)*A*ta).T*ta)/(((ta.T)*ta)[0])))[0]

Theorem 1.4 in [here](https://www.proquest.com/openview/95bfe6ed9987b6718ad3feb95a1480bd/1?pq-origsite=gscholar&cbl=18750&diss=y) implies that since the bottom inclusion is periodic then there exists $n_1$ such that for all $n\geq n_1$ we have $$G_{0,n}G_{0,n}^t s_n = [A_{1,\infty}:A_{0,\infty}] s_n$$
where 
$$G_{0,n}G_{1,n} = \begin{cases}G&,n\text{ even}\\
G^t&,n\text{ odd}\end{cases}$$
and $[A_{1,\infty}:A_{0,\infty}]=\|s_n\|^2/\|t_n\|^2$. To avoid dealing with the trace vector $t_n$ of the middle inclusions, we will compute
$$\frac{\langle G_{0,n}G_{0,n}^t s_n,s_n\rangle}{\|s_n\|^2}$$
and determine if there is any pair $(G_{0,n},G_{1,n})$ for which this is equal to $\frac{3+\sqrt{5}}{2}$.

In [5]:
threshold = (3+np.sqrt(5))/2
threshold

2.618033988749895

Using our building block decomposition we form all pairs $(G_{0,n},G_{1,n})$, eliminating isomorphic pairs at every step.

In [ ]:
# no longer needed
# Gf = np.linalg.multi_dot(result[0])
# af_0 = np.ones((5,1))
# eigvals, eigvects = np.linalg.eig(Gf@np.transpose(Gf))
# sf_0_raw = -eigvects[:,0]
# sf_0 = sf_0_raw/(sf_0_raw@af_0)

# def af(n):
#     return (np.linalg.matrix_power(Gf@np.transpose(Gf),n))@af_0

# def sf(n):
#     return sf_0/(sf_0@af(n))

# def norm_lim(A,n):
#     s = sf(n)
#     return (((A@np.transpose(A))@s)@s)/(np.linalg.norm(s)**2)

In [ ]:
# no longer needed
# candidates = []
# tol = 0.5

# for list in result:
#     r = len(list)
#     for i in range(2,r-1):
#         G1f = np.linalg.multi_dot(list[:i])
#         G2f = np.linalg.multi_dot(list[i:])
#         if threshold-tol<norm_lim(G1f,0)<threshold:
#             candidates.append([G1f,G2f])


In [90]:
from graph_fact_tools import *

candidates_abs = []
for list in result:
    r = len(list)
    G1f = list[:1][0]
    G2f = np.linalg.multi_dot(list[1:])
    candidates_abs.append([G1f,G2f])
    G1f = np.linalg.multi_dot(list[:10])
    G2f = list[10:][0]
    candidates_abs.append([G1f,G2f])
    for i in range(2,r-1):
        G1f = np.linalg.multi_dot(list[:i])
        G2f = np.linalg.multi_dot(list[i:])
        candidates_abs.append([G1f,G2f])
    candidates_abs = dedupe_chains(candidates_abs)
print(len(candidates_abs))

35


We obtained precisely $35$ pairs. Now, transform our candidate list to a list of symbolic matrices and compute their norms symbolically.

In [152]:
candidates_sym = [[sp.Matrix(pair[0]),sp.Matrix(pair[1])] for pair in candidates_abs]
norm_list = [[norm_sym(pair[0],0),norm_sym2(pair[1],0)] for pair in candidates_sym]

Below we have the list of norm pairs for all of our unique factorizations.

In [153]:
from collections import Counter

# convert to numeric (higher precision) then round for grouping
pairs_num = []
rep_sym = {}
for a_sym, b_sym in norm_list:
    af = float(sp.N(a_sym, 24))
    bf = float(sp.N(b_sym, 24))
    key = (round(af, 8), round(bf, 8))   # adjust precision as needed
    pairs_num.append(key)
    rep_sym.setdefault(key, (a_sym, b_sym))

cnt = Counter(pairs_num)

rows = []
for (n1, n2), c in sorted(cnt.items()):
    a_sym, b_sym = rep_sym[(n1, n2)]
    prod_sym = latex(sp.together(simplify(a_sym*b_sym)))
    a_tex = latex(sp.together(simplify(a_sym)))
    b_tex = latex(sp.together(simplify(b_sym)))
    rep_tex = f"{a_tex} & {b_tex}"          # math-mode representative
    rows.append(f"{n1:.8f} & {n2:.8f} & {c:d} & {rep_tex} & {prod_sym}\\\\")

# header + join
header = r"\text{Norm 1} & \text{Norm 2} & \text{count} & \|G_{0,n}\|^2 & \|G_{1,n}\|^2 & \text{Product}\\ \hline"
body = "\n".join(rows)
array_tex = r"\begin{array}{r r r c c c}" + "\n" + header + "\n" + body + "\n" + r"\end{array}"

display(Math(array_tex))

<IPython.core.display.Math object>

None of them have norm $\frac{3+\sqrt{5}}{2}$.